# 5. 合併 LoRA 權重並儲存模型

在本實驗的最後一個環節，我們將學習 `peft` 函式庫的一個非常實用的功能：將訓練好的 LoRA adapter 權重合併回原始的基礎模型中。

這樣做的好處是，我們可以得到一個**單一的、標準的 Hugging Face 模型**，它已經包含了微調後的所有知識。這個合併後的模型可以像任何普通的 `transformers` 模型一樣被載入和使用，不再需要 `peft` 函式庫，這對於簡化部署流程非常方便。

## 步驟 1: 載入基礎模型與 LoRA Adapter

和上一個筆記本一樣，我們首先需要載入量化後的基礎模型，以及我們訓練好的 LoRA adapter。


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

# --- 重新載入基礎模型與 Tokenizer ---
model_id = "meta-llama/Llama-2-7b-chat-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# --- 載入 LoRA Adapter ---
output_dir = "./lora-llama2-7b-chat"
latest_checkpoint = max([os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint-")], key=os.path.getmtime)
peft_model = PeftModel.from_pretrained(base_model, latest_checkpoint)

print("模型與 Adapter 載入完成。")




## 步驟 2: 合併權重

`peft` 提供了一個非常簡單的方法來合併權重：`merge_and_unload()`。

這個方法會執行以下操作：
1.  將 LoRA adapter (矩陣 A 和 B) 的權重計算出來 (`B * A`)。
2.  將計算出的權重加回到原始的 `target_modules` (例如 "q_proj", "v_proj") 上。
3.  從模型中移除 LoRA adapter，並返回一個合併後的標準 `transformers` 模型。

**注意**: 由於我們使用了量化模型，`merge_and_unload()` 目前可能無法直接在量化層上運作。在理想情況下（非量化模型），這個步驟會非常直接。對於量化模型，`peft` 函式庫正在持續改進支援。這裡我們展示標準流程。


In [ ]:
# 合併權重
# 在非量化模型上，這會直接返回一個 AutoModelForCausalLM 物件
merged_model = peft_model.merge_and_unload()

print("權重合併完成！")
# 合併後，模型類型變回原本的 LlamaForCausalLM
print(type(merged_model))


## 步驟 3: 儲存完整模型與 Tokenizer

合併後的模型現在是一個完整的、獨立的模型。我們可以使用 `save_pretrained()` 方法將它和它的 Tokenizer 一起儲存到一個新的目錄中。

這個儲存下來的模型未來可以直接使用 `AutoModelForCausalLM.from_pretrained()` 載入，不再需要 `peft` 函式庫。


In [ ]:
# 定義儲存目錄
save_directory = "./llama2-7b-chat-guanaco-merged"

# 儲存模型
merged_model.save_pretrained(save_directory)

# 儲存 Tokenizer
tokenizer.save_pretrained(save_directory)

print(f"合併後的模型已儲存至: {save_directory}")


## 實驗總結

恭喜您完成了整個 PEFT 微調實驗！

您已經學會了：
- **環境設定**: 準備進行 PEFT 的開發環境。
- **載入與量化**: 如何載入大型語言模型並使用 4-bit 量化來節省資源。
- **應用 LoRA**: 如何使用 `peft` 函式庫設定並應用 LoRA。
- **訓練**: 如何使用 `Trainer` 進行微調。
- **推理**: 如何使用微調後的模型生成文本。
- **合併與儲存**: 如何將 LoRA 權重合併回主模型，以便於部署。

希望這個實驗能幫助您掌握參數高效微調的核心技術，並將其應用到您自己的專案中！
